In [1]:
import pandas as pd
import numpy as np

In [2]:
link='https://en.wikipedia.org/wiki/List_of_American_films_of_2020'

In [3]:
df1=pd.read_html(link,header=0)[2]
df2=pd.read_html(link,header=0)[3]
df3=pd.read_html(link,header=0)[4]
df4=pd.read_html(link,header=0)[5]

In [4]:
df=df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_3272\3617246550.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [5]:
df=df.drop(['Opening','Opening.1','Production company','.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.','Ref.'],axis=1)

In [6]:
#director
df['director_name']=df['Cast and crew'].str.split('(',expand=True)[0]

In [7]:
#actors
df['actors']=df['Cast and crew'].str.split(';',expand=True)[1]
df['actors1']=df['Cast and crew'].str.split(';',expand=True)[2]

In [8]:
df['actors1']=df['actors1'].fillna('Unknown')

In [9]:
index=df[df['actors1']=='Unknown'].index

In [10]:
index=list(index)

In [11]:
for i in index:
    df['actors1'][i]=df['actors'][i]

In [12]:
df=df.drop('actors',axis=1)

In [13]:
df['actor_1_name']=df['actors1'].str.split(',',expand=True)[0]
df['actor_2_name']=df['actors1'].str.split(',',expand=True)[1]
df['actor_3_name']=df['actors1'].str.split(',',expand=True)[2]

In [14]:
df=df.drop(['actors1','Cast and crew'],axis=1)

In [15]:
df.rename(columns={'Title':'movie_title'},inplace=True)

In [16]:
df

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...
270,We Can Be Heroes,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
271,News of the World,Paul Greengrass,Tom Hanks,Helena Zengel,None
272,One Night in Miami...,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
273,Promising Young Woman,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [31]:
df=df[df['movie_title']!='No films had been released on this date as resulting the closure of theaters caused by the COVID-19 pandemic']

In [44]:
df[df['movie_title']=='The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)']

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name


In [43]:
df['movie_title']=df['movie_title'].replace('The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)','The SpongeBob Movie: Sponge on the Run')

C:\Users\HP\AppData\Local\Temp\ipykernel_3272\3864763729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['movie_title']=df['movie_title'].replace('The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)','The SpongeBob Movie: Sponge on the Run')


In [45]:
#genre
from tmdbv3api import TMDb
import requests
import json
from tmdbv3api import Movie
tmdb=TMDb()
tmdb.api_key='e770358dbe349592b1fba6cb607230fb'

In [46]:
from tmdbv3api import Movie
tmdb_movie=Movie()
def get_genre(x):
    genres=[]
    result=tmdb_movie.search(x)
    movie_id=result[0].id
    response=requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json=response.json()
    if data_json['genres']:
        genre_str=' '
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            np.NaN

In [47]:
df['genres']=df['movie_title'].map(lambda x:get_genre(str(x)))

C:\Users\HP\AppData\Local\Temp\ipykernel_3272\473933908.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres']=df['movie_title'].map(lambda x:get_genre(str(x)))


In [49]:
df=df[['movie_title','genres','director_name','actor_1_name','actor_2_name','actor_3_name']]

In [51]:
df['movie_title']=df['movie_title'].str.lower()

In [52]:
df.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,the grudge,Horror,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,underwater,Action,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,like a boss,Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,three christs,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,inherit the viper,Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs


In [53]:
df.isna().sum()

movie_title       0
genres            1
director_name     0
actor_1_name      2
actor_2_name      6
actor_3_name     27
dtype: int64

In [54]:
df.fillna('Unknown',inplace=True)

In [55]:
df.isnull().sum()

movie_title      0
genres           0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
dtype: int64

In [56]:
df['comb']=df['actor_1_name']+' '+df['actor_2_name']+' '+df['actor_3_name']+' '+df['director_name']+' '+df['genres']

In [57]:
df

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,the grudge,Horror,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Andrea Riseborough Demián Bichir John Cho N...
1,underwater,Action,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Kristen Stewart Vincent Cassel Jessica Henw...
2,like a boss,Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Tiffany Haddish Rose Byrne Salma Hayek Migu...
3,three christs,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Richard Gere Peter Dinklage Walton Goggins ...
4,inherit the viper,Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Josh Hartnett Margarita Levieva Chandler Ri...
...,...,...,...,...,...,...,...
270,we can be heroes,Action,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Priyanka Chopra Jonas Pedro Pascal YaYa Gos...
271,news of the world,Drama,Paul Greengrass,Tom Hanks,Helena Zengel,Unknown,Tom Hanks Helena Zengel Unknown Paul Greengr...
272,one night in miami...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Kingsley Ben-Adir Eli Goree Aldis Hodge Reg...
273,promising young woman,Thriller,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Carey Mulligan Bo Burnham Alison Brie Emera...


In [58]:
data=pd.read_csv('movie_data.csv')

In [59]:
data.shape

(5882, 8)

In [60]:
data=data.append(df,ignore_index=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_3272\1484680574.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df,ignore_index=True)


In [62]:
data=data.drop('Unnamed: 0',axis=1)

In [64]:
data.to_csv('movie_data.csv')

In [65]:
data.shape

(6156, 7)